In [12]:
import win32com.client
import os
import shutil
import pandas as pd
import numpy as np
from datetime import datetime
path = '//NBNOTOOPS1/opspublic/Trade Support Desk (Cash)/ETF Client Confirmations/'
files = [f for f in os.listdir(path+'Confirm Emails/') if '.msg' in f]
#print (files)
if os.path.isdir("//NBNOTOOPS1/opspublic/Trade Support Desk (Cash)/ETF Client Confirmations/Attachments/" + datetime.today().strftime('%Y%m%d')):
    shutil.rmtree("//NBNOTOOPS1/opspublic/Trade Support Desk (Cash)/ETF Client Confirmations/Attachments/" + datetime.today().strftime('%Y%m%d'))
os.mkdir("//NBNOTOOPS1/opspublic/Trade Support Desk (Cash)/ETF Client Confirmations/Attachments/" + datetime.today().strftime('%Y%m%d'))
for file in files:
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    msg = outlook.OpenSharedItem(os.path.join(path+'Confirm Emails/', file))
    att=msg.Attachments
    for i in att:
        try:
            filename, file_extension = str(i).rsplit('.', 1)
            if file_extension.lower() == 'csv' or file_extension.lower() == 'xls' or file_extension.lower() == 'xlsx':
                i.SaveAsFile(os.path.join(path+'Attachments/' + datetime.today().strftime('%Y%m%d'), i.FileName))
        except:
            None
    att = None
    msg = None
    outlook = None

print ('Files not read:')
ticker_list=[]
qty_list=[]
net_list=[]
cr_list = []
directory = '//NBNOTOOPS1/opspublic/Trade Support Desk (Cash)/ETF Client Confirmations/Attachments/' + datetime.today().strftime('%Y%m%d') + '/'
for filename in os.listdir(directory):
    name, ext = os.path.splitext(filename)
    if ext == '.csv':
        file = pd.read_csv(directory+filename, index_col=None, header=None)    
    else:
        file = pd.read_excel(directory+filename, index_col=None, header=None)
    ## Searching for Ticker ##
    a = None
    a = np.where(file.values == 'FUND_TICKER')
    if not a[0].size > 0:
        a = np.where(file.values == 'Fund Ticker')
    if not a[0].size > 0:
        a = np.where(file.values == 'FUND TICKER')
    if not a[0].size > 0:
        a = np.where(file.values == 'Ticker:')

    if a[0].size > 0 and str(file.iloc[a[0],a[1]].values[0][0]) == 'Ticker:':
        i=a[1][0]+1
        while pd.isnull(file.iloc[a[0],a[1]+i].values[0][0]):
            i+=1            
        ticker = file.iloc[a[0],a[1]+i].values[0][0]
    elif a[0].size > 0:
        ticker = file.iloc[a[0],a[1]+1].values[0][0]

    if a[0].size > 0:
    ## Searching for Quantity ##
        q = None
        q = np.where(file.values == 'TOTAL_ETF_SHARES')
        if not q[0].size > 0:
            q = np.where(file.values == 'TRADE SIZE')
        if not q[0].size > 0:
            i = int(0)
            for item in file.values:
                j = int(0)
                for item2 in file.values[i]:
                    if 'Quantity' in str(item2):
                        q = [np.array([i]),np.array([j])]
                        break
                    else:
                        j+=1
                else:
                    i+=1
        if not q[0].size > 0:
            q = np.where(file.values == 'Units Created')
        if not q[0].size > 0:
            q = np.where(file.values == 'Units Redeemed')
        if not q[0].size > 0:
            q = np.where(file.values == 'Shares')
        i=1
        while pd.isnull(file.iloc[q[0],q[1]+i].values[0][0]):
            i+=1
        qty = file.iloc[q[0],q[1]+i].values[0][0]

    ## Searching for NET Cash ##
        n = None
        n = np.where(file.values == 'TOTAL_DUE')
        if not n[0].size > 0:
            n = np.where(file.values == 'Amount of Cash Component of Purchase:')
        if not n[0].size > 0:
            n = np.where(file.values == 'Amount of Cash Component of Redemption:')
        if not n[0].size > 0:
            n = np.where(file.values == 'Total Cash Amount Due/Owed AP')
        if not n[0].size > 0:
            n = np.where(file.values == 'TOTAL CASH')
        if not n[0].size > 0:
            n = np.where(file.values == 'Total Net Amount:')
        if not n[0].size > 0:
            n = np.where(file.values == 'Total Net Amount')
        if not n[0].size > 0:
            n = np.where(file.values == 'Total Net Value')
        if n[0].size > 0 and file.iloc[n[0],n[1]].values[0][0] == 'TOTAL CASH':
            net = file.iloc[n[0]+1,n[1]].values[0][0]
        else:
            i=1
            while pd.isnull(file.iloc[n[0],n[1]+i].values[0][0]):
                i+=1
            net = file.iloc[n[0],n[1]+i].values[0][0]
        
    ## Searching for Create or Redemption ##
        cr = None
        i = int(0)
        for item in file.values:
            j = int(0)
            for item2 in file.values[i]:
                if item2 != 'SUBSCRIPTION/REDEMPTION CONFIRMATION':
                    if any(s in str(item2).lower() for s in ['create', 'creation', 'subscription']):
                        cr = 'Creation'
                        break
                    elif any(s in str(item2).lower() for s in ['redeem', 'redemption']):
                        cr = 'Redemption'
                        break
                    else:
                        j+=1
                else:
                    if float(qty) > 0: cr = 'Creation' 
                    else: cr = 'Redemption'
            else:
                i+=1
        if cr == None: cr = 'No info'     
            
        ticker_list.append(ticker)
        qty_list.append(float(qty))
        net_list.append(float(net))
        cr_list.append(cr)

    else:
        print(filename)

df = pd.DataFrame({'Ticker': ticker_list,
                   'Qty' : qty_list,
                   'NET' : net_list,
                   'Type': cr_list
                 })
df['Ticker'] = df['Ticker'].str.replace("'","")

# Moving .msg files to Old Confirms folder 
for file in files:
    if os.path.isfile(os.path.join(path+'Old confirm Emails/', file)):
        os.remove(os.path.join(path+'Old confirm Emails/', file))
    else: shutil.move(os.path.join(path+'Confirm Emails/', file), os.path.join(path+'Old confirm Emails/', file))

# Loading ETF Uploader spreadsheet
file = pd.read_excel('//NBNOTOOPS1/opspublic/Trade Support Desk (Cash)/ETF IMAGINE ADJ T+1/Saved Uploads/ETF UPLOADER T+1 ADJ ' + datetime.today().strftime('%Y%m%d') + ' 1.xlsm', skiprows = 99)
split = file['Sec Symbol'].str.split(".", expand = True)
file = file.loc[~file['Comment (Order)'].str.startswith("ADJ_")]
file['Symbol'] = split[0].str.strip()
file['Net Cost'] = file['Quantity'] * file['Cost']
file
df1 = df
df1['Ticker'] = df1['Ticker'].str.strip()
symbol_imagine = ['_u', 'a', 'b', 'c', 'f', 'l', 'u', 'v', 'x']
symbol_adp = ['.UN', '.A', '.B', '.C', '.F', '.L', '.U', '.V', '.X']
for i, j in zip(symbol_imagine, symbol_adp):
    file['Symbol'] = file['Symbol'].str.replace(i, j, regex=False)
# Merging both DFs
compare = pd.merge(left=df1, right=file[['Symbol', 'Quantity', 'Net Cost']], how='outer', left_on='Ticker', right_on='Symbol')
for i in range(len(compare)):
    if compare['Type'][i] == 'Redemption' and compare['Qty'][i] > 0:
        compare['Qty'].iloc[i] = compare['Qty'][i] * -1
        if compare['NET'][i] > 0: compare['NET'].iloc[i] = compare['NET'][i] * -1
        else: compare['NET'].iloc[i] = 0
    elif compare['Type'][i] == 'Redemption' and compare['NET'][i] > 0: compare['NET'].iloc[i] = 0
    elif compare['Type'][i] == 'Creation' and compare['NET'][i] < 0: compare['NET'].iloc[i] = 0

compare['Difference in QTY'] = compare['Qty'].sub(compare['Quantity'],fill_value=0)
compare['Difference in Money'] = compare['NET'].sub(compare['Net Cost'],fill_value=0).round(2)
compare.to_excel ('//NBNOTOOPS1/opspublic/Trade Support Desk (Cash)/ETF Client Confirmations/ETFs List ' + datetime.today().strftime('%Y%m%d') + '.xls')

compare

Files not read:
